In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import time
from collections import OrderedDict
from dtw import dtw
import xml.etree.ElementTree
motion_sensors = [1, 10, 11, 12, 13, 14, 2, 3, 4, 5, 6, 7, 8, 9]
global sensors_list

sensors_list = []

In [2]:
adj_matrix = {"bedroom" : set(['diningroom', 'bathroom']), 
              "bathroom" : set(['diningroom', 'bedroom']),
              "diningroom" : set(['bathroom', 'livingroom', 'bedroom', 'kitchen']),
              "livingroom" : set(['diningroom']),
              "kitchen" : set(['diningroom'])
             }

def smoothListGaussian(list,strippedXs=False,degree=2):  
    window=degree*2-1  
    weight=np.array([1.0]*window)  
    weightGauss=[]  
    for i in range(window):  
        i=i-degree+1  
        frac=i/float(window)  
        gauss=1/(np.exp((4*(frac))**2))  
        weightGauss.append(gauss)  

    weight=np.array(weightGauss)*weight  
    smoothed=[0.0]*(len(list)-window)  
    
    for i in range(len(smoothed)):  
        smoothed[i]=sum(np.array(list[i:i+window])*weight)/sum(weight)  
        if (smoothed[i] < 0.1):
            smoothed[i] = 0
    
    return smoothed      

def room_factor(graph, start, end, path=[]):
    path = path + [start]
    # print(start)
    if start not in graph:     return None
    elif start == end:         return 1
    elif end in graph[start]:  return 2
    else:                      return 3 #4

def FiringSequence(l):
    seq = []
    inds = []
    prev_item = 0
    for i in range(len(l)):
        if (not l[i] == prev_item):
            seq.append(l[i])
            inds.append(i)
            prev_item = l[i]
    return seq, inds
        
def get_sensors_activation_sequence(readings, data_type, df_):
    sensor_seq = []
    for i in range(len(readings)):
        zero_readings = True
        d = ast.literal_eval(readings[i])
        for ind in range(len(d)):
            if (d[ind] == 1):
                zero_readings = False
                t = df_.time[i]
                x = motion_sensors[ind]
                if (x < 9 or x > 12):
                    if (data_type == "synt"):
                        x += 0.2
                    sensor_seq.append((t, x))
        if (zero_readings == True):
            t = df_.time[i]
            x = -1
            sensor_seq.append((t, x))
            
    return sensor_seq

def GeoDist(s1, s2):
    if (s1 == -1 and s2 == -1): return 0
    if (s1 == -1 or s2 == -1): return 100
    for sensor in sensors_list:
        if int(sensor.sensor_id) == s1:
            sensor1 = sensor
            
    for sensor in sensors_list:
        if int(sensor.sensor_id) == s2:
            sensor2 = sensor
            
    p1 = np.asarray([float(int(sensor1.x) / 100), float(int(sensor1.y) / 100)])
    p2 = np.asarray([float(int(sensor2.x) / 100), float(int(sensor2.y) / 100)])
    # print(np.linalg.norm(p1 - p2))
    # print(room_factor(adj_matrix, sensor1.room, sensor2.room))
    
    return np.linalg.norm(p1 - p2) * room_factor(adj_matrix, sensor1.room, sensor2.room)

def GeoDist2(s1, s2):
    if (s1 == s2): return 0
    else: return 1

def Sequence_analyzer(UltimateDatasetName, session_info):
    global real_sensor_seq
    real_sensor_seq = []
    global synt_sensor_seq
    synt_sensor_seq = []
    global synt_indices
    global real_indices
    global df_
    global time_list_formatted
    
    df_ = (pd.read_csv(UltimateDatasetName + ".csv"))
    time_list_formatted = [time.strftime('%H:%M:%S', time.localtime(x)) for x in df_.time]

    # print(len(df_))
    real_data = np.asarray(get_sensors_activation_sequence(df_.gt_motion_readings, "real",df_))
    synt_data = np.asarray(get_sensors_activation_sequence(df_.synthetic_motion_readings, "synt", df_))

    # fig = plt.figure()
    # fig, ax = plt.subplots(figsize = (10, 7))
    # ax = fig.add_subplot(1, 1, 1)
    
    # plt.gca().set_ylim([0,15])
    # plt.scatter(real_data[:,0], real_data[:,1], alpha=0.8, c="b", edgecolors='none', s=30, label="Real-World Dataset")
    # plt.scatter(synt_data[:,0], synt_data[:,1], alpha=0.8, c="r", edgecolors='none', s=30, label="Synthetic Dataset")
    # plt.xticks(np.arange(0, len(time_list_formatted), step=20), time_list_formatted[::20])
    # ax.set_xticklabels(time_list_formatted[::int(len(time_list_formatted)/8)])
    # plt.yticks(range(1, 15))
    # plt.xticks(rotation=90)
    # plt.xlabel("time (HH:MM:SS)")
    # plt.ylabel("Sensor Number")
    # plt.title('Sensors Activation Sequence (' + session_info + ')')
    # plt.grid(color='k', linestyle=':', linewidth=0.5)
    # plt.legend(loc=1)
    # plt.show()
    
    # real_sensor_seq, real_indices = FiringSequence(real_data[:,1])
    # synt_sensor_seq, synt_indices = FiringSequence(list(map(int, synt_data[:,1])))
    
    real_sensor_seq = real_data[:,1]
    synt_sensor_seq = list(map(int, synt_data[:,1]))

In [3]:
############################################
####      SENSOR CLASSES DEFINITION     ####
############################################

class Sensor:
    sensor_type = ""
    sensor_id = ""
    x = float()
    y = float()
    z = float()
    r = float()
    room = str()

    def __str__(self):
        return 'type: %s,\n id: %s,\n x: %s,\n y: %s\n'%(self.sensor_type, self.sensor_id, self.x, self.y)

    def Initialize(self, element):
        self.x = str(660 - int(str(element.find('point').find('xcoord').text)))
        self.y = (str(element.find('point').find('ycoord').text))
        self.sensor_id = (str(element.find('id').text)).replace('sensor','')
        self.sensor_type = (str(element.find('type').text))     
        return 0
    
    def GaussianFiredSensor(self, simulated_localization, real_localization):
        import math
        sensor_location = [float(self.x)/10, float(self.y)/10]
        cov = [[self.sensing_area, 0],[0, self.sensing_area]]        
        prob = multivariate_normal(simulated_localization, cov)
        probCdf = prob.cdf(sensor_location)
        return probCdf
    
class MotionSensorBinary(Sensor):
    def __init__(self, element):
        Sensor.Initialize(self, element)
        self.sensing_area = (str(element.find('radius').text))
        self.room = (str(element.find('room').text))

In [4]:
############################################
####       PARSE SIMULATION WORLD       ####
############################################

def ParseSimulationWorld(simworldname):
    root = xml.etree.ElementTree.parse(simworldname).getroot()
    global sensors_list
    for element in root:
        if(element.tag == 'sensors'):
            for sensor in element:
                #Choose the kind of sensor we have
                if(str(sensor.find('type').text) == 'RFID_Reader'):
                    # this_sensor = RFID_Reader(sensor)
                    continue

                if(str(sensor.find('type').text) == 'MotionSensorBinary'):
                    # this_sensor = MotionSensorBinary(sensor, radius)
                    this_sensor = MotionSensorBinary(sensor)

                if(str(sensor.find('type').text) == 'MotionSensorAnalog'):
                    this_sensor = MotionSensorAnalog(sensor)

                if(str(sensor.find('type').text) == 'PressureSensorAnalog'):
                    # this_sensor = PressureSensorAnalog(sensor)
                    continue

                if(str(sensor.find('type').text) == 'BeaconSensor'):
                    #this_sensor = BeaconSensor(sensor)
                    continue

                sensors_list.append(this_sensor)

In [5]:
def FindClosestDataPoint(item, reference_list, lower_limit, upper_limit):
    temp_list = reference_list.copy()
    lower_limit = int(lower_limit)
    upper_limit = int(upper_limit)
    ind = lower_limit
    ambig = []
    
    diff = GeoDist(int(item), int(temp_list[lower_limit]))
    for i in range(lower_limit, upper_limit):
        ambig.append(temp_list[i])
        new_diff = GeoDist(int(item), int(temp_list[i]))
        if (new_diff < diff):
            diff = new_diff
            ind = i
        if (diff == 0):
            break
    return ind, set(ambig)

def SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq, window_size):
    matching_list = []
    ambigs = []
    speed = len(synt_sensor_seq) / len(real_sensor_seq)
    j = 0
    for i in range(len(real_sensor_seq)):
        if (not (real_sensor_seq[i] == -1)):
            if (j - window_size < 0): lower_limit = 0
            else: lower_limit = j - window_size
            if (j + window_size > len(synt_sensor_seq)): upper_limit = len(synt_sensor_seq)
            else: upper_limit = j + window_size

            ind, ambig = FindClosestDataPoint(real_sensor_seq[i], synt_sensor_seq, lower_limit, upper_limit)
            matching_list.append(synt_sensor_seq[ind])
            ambigs.append(ambig)
        j = (i+1) * speed
    return matching_list, ambigs

In [6]:
def Rmse(UltimateDatasetNames, titles, max_half_window_size):
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    rmses = {}
    matching_list_dict = {}
    ambings_dict = {}
    global real_data_dict
    real_data_dict = {}
    for index in range(len(UltimateDatasetNames)):
        Sequence_analyzer(UltimateDatasetNames[index], titles[index])
        matching_list_dict[UltimateDatasetNames[index]] = {}
        ambings_dict[UltimateDatasetNames[index]] = {}
        real_data_dict[UltimateDatasetNames[index]] = {}
        rmses[titles[index]] = []

        for window_size in range(1, max_half_window_size + 1):
            if (window_size % 10 == 0):
                print(window_size * 100 / max_half_window_size, ' % completed')
            if (UltimateDatasetNames[index] == "Data//Results//Ultimate_Dataset_28jun2"):
                matching_list, ambigs = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq[:1510], window_size)
                matching_list_dict[UltimateDatasetNames[index]][window_size] = matching_list
                ambings_dict[UltimateDatasetNames[index]][window_size] = ambigs
                rmses[titles[index]].append(sqrt(mean_squared_error(list(filter(lambda a: a != -1, real_sensor_seq[:1510])), matching_list)))
                # rmses[titles[index]].append(sqrt(mean_squared_error(real_sensor_seq[:1510], matching_list)))
            else:
                matching_list, ambigs = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq, window_size)
                matching_list_dict[UltimateDatasetNames[index]][window_size] = matching_list
                ambings_dict[UltimateDatasetNames[index]][window_size] = ambigs
                rmses[titles[index]].append(sqrt(mean_squared_error(list(filter(lambda a: a != -1, real_sensor_seq)), matching_list)))
                # rmses[titles[index]].append(sqrt(mean_squared_error(real_sensor_seq, matching_list)))

        real_data_dict[UltimateDatasetNames[index]]['time'] = time_list_formatted
        real_data_dict[UltimateDatasetNames[index]]['data'] = real_sensor_seq
    
    return rmses, matching_list_dict, ambings_dict

In [7]:
def Recall(UltimateDatasetNames, titles, max_half_window_size):
    from sklearn.metrics import recall_score
    recalls = {}
    for index in range(len(UltimateDatasetNames)):
        Sequence_analyzer(UltimateDatasetNames[index], titles[index])
        matching_list_dict = {}
        recalls[titles[index]] = []

        for window_size in range(1, max_half_window_size + 1):
            if (window_size % 10 == 0):
                print(window_size * 100 / max_half_window_size, ' % completed')
            if (UltimateDatasetNames[index] == "Data//Results//Ultimate_Dataset_28jun2"):
                matching_list = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq[:1510], window_size)
                matching_list_dict[window_size] = matching_list
                recalls[titles[index]].append(recall_score(list(filter(lambda a: a != -1, real_sensor_seq[:1510])), matching_list, average='micro'))
            else:
                matching_list = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq, window_size)
                matching_list_dict[window_size] = matching_list
                recalls[titles[index]].append(recall_score(list(filter(lambda a: a != -1, real_sensor_seq)), matching_list, average='micro'))
                
    return recalls

In [8]:
def Precision(UltimateDatasetNames, titles, max_half_window_size):
    from sklearn.metrics import precision_score
    precisions = {}
    for index in range(len(UltimateDatasetNames)):
        
        Sequence_analyzer(UltimateDatasetNames[index], titles[index])
        matching_list_dict = {}
        precisions[titles[index]] = []

        for window_size in range(1, max_half_window_size + 1):
            if (window_size % 10 == 0):
                print(window_size * 100 / max_half_window_size, ' % completed')
            if (UltimateDatasetNames[index] == "Data//Results//Ultimate_Dataset_28jun2"):
                matching_list = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq[:1510], window_size)
                matching_list_dict[window_size] = matching_list
                precisions[titles[index]].append(precision_score(list(filter(lambda a: a != -1, real_sensor_seq[:1510])), matching_list, average='micro'))
            else:
                matching_list = SlidingWindowDataMatching(synt_sensor_seq, real_sensor_seq, window_size)
                matching_list_dict[window_size] = matching_list
                precisions[titles[index]].append(precision_score(list(filter(lambda a: a != -1, real_sensor_seq)), matching_list, average='micro'))
                
    return precisions

In [9]:
def Print(D, titles, metric, max_half_window_size):
    fig, ax = plt.subplots(figsize = (9, 5))
    styles = ['-', '--', ':', '-.']
    for i in range(len(titles)):
        plt.plot([(i * 6)/60 for i in range(1, max_half_window_size + 1)], D[titles[i]], styles[i], color = 'k')
    
    plt.grid(color='k', linestyle=':', linewidth=0.5)
    # plt.subplots(figsize = (9, 5))
    plt.xticks(np.arange(0, 11, 1.0))
    plt.xlabel("Window Size (Minutes)")
    plt.ylabel(metric)
    
    plt.legend(titles)
    plt.show()
    
def PrintAmbigs(A, UltimateDatasetNames, max_half_window_size, titles):
    fig, ax = plt.subplots(figsize = (9, 5))
    styles = ['-', '--', ':', '-.']
    for i in range(len(UltimateDatasetNames)):
        Xs = []
        for w in range(1, max_half_window_size + 1):
            X = 0
            for item in range(len(A[UltimateDatasetNames[i]][w])):
                X += len(A[UltimateDatasetNames[i]][w][item])
            
            Xs.append(X / len(A[UltimateDatasetNames[i]][w]))
            
        plt.plot([(i * 6)/60 for i in range(1, max_half_window_size + 1)], Xs, styles[i], color = 'k')
        
    plt.ylabel("Average Ambiguity")
    plt.xlabel("Window Size (Minutes)")
    plt.legend(titles)
    plt.grid(color='k', linestyle=':', linewidth=0.5)
    


In [10]:
UltimateDatasetName1 = "Data//Results//Ultimate_Dataset_28jun1"
title1 = "28 June - 1st Session"
UltimateDatasetName2 = "Data//Results//Ultimate_Dataset_28jun2"
title2 = "28 June - 2nd Session"
UltimateDatasetName3 = "Data//Results//Ultimate_Dataset_4jul1"
title3 = "4 July - 1st Session"
UltimateDatasetName4 = "Data//Results//Ultimate_Dataset_4jul2"
title4 = "4 July - 2nd Session"
simworldname = 'Data//simulationWorld2.xml'

UltimateDatasetNames = [UltimateDatasetName1, UltimateDatasetName2, 
                        UltimateDatasetName3, UltimateDatasetName4]
titles = [title1, title2, title3, title4]
max_half_window_size = 150

ParseSimulationWorld(simworldname)
print("RMSE CALCULATION")
rmses, matching_list_dict, ambigs_list = Rmse(UltimateDatasetNames, titles, max_half_window_size)

# print("RECALL CALCULATION")
# recalls = Recall(UltimateDatasetNames, titles, max_half_window_size, '->')
# print("PRECISION CALCULATION")
# precisions = Precision(UltimateDatasetNames, titles, max_half_window_size, '->')

# print("(REVERSE) RMSE CALCULATION")
# r_rmses = Rmse(UltimateDatasetNames, titles, max_half_window_size, '<-')
# print("(REVERSE) RECALL CALCULATION")
# r_recalls = Recall(UltimateDatasetNames, titles, max_half_window_size, '<-')
# print("(REVERSE) PRECISION CALCULATION")
# r_precisions = Precision(UltimateDatasetNames, titles, max_half_window_size, '<-')

RMSE CALCULATION
6.666666666666667  % completed
13.333333333333334  % completed
20.0  % completed
26.666666666666668  % completed
33.333333333333336  % completed
40.0  % completed
46.666666666666664  % completed
53.333333333333336  % completed
60.0  % completed
66.66666666666667  % completed
73.33333333333333  % completed
80.0  % completed
86.66666666666667  % completed
93.33333333333333  % completed
100.0  % completed
6.666666666666667  % completed
13.333333333333334  % completed
20.0  % completed
26.666666666666668  % completed
33.333333333333336  % completed
40.0  % completed
46.666666666666664  % completed
53.333333333333336  % completed
60.0  % completed
66.66666666666667  % completed
73.33333333333333  % completed
80.0  % completed
86.66666666666667  % completed
93.33333333333333  % completed
100.0  % completed
6.666666666666667  % completed
13.333333333333334  % completed
20.0  % completed
26.666666666666668  % completed
33.333333333333336  % completed
40.0  % completed
46.66666

KeyboardInterrupt: 

In [ ]:
Print(rmses, titles, 'RMSE', max_half_window_size)
PrintAmbigs(ambigs_list, UltimateDatasetNames, max_half_window_size, titles)

# Print(recalls, titles, 'Recall', max_half_window_size)
# Print(precisions, titles, 'Precision', max_half_window_size)

# Print(r_rmses, titles, 'RMSE', max_half_window_size)
# Print(r_recalls, titles, 'Recall', max_half_window_size)
# Print(r_precisions, titles, 'Precision', max_half_window_size)

In [ ]:
for session in UltimateDatasetNames:
    dist, cost_matrix, acc_cost_matrix, path = dtw(matching_list_dict[session][4], 
                                                   real_data_dict[session]['data'],
                                                   dist=GeoDist, 
                                                   w = abs(len(real_data_dict[session]['data']) - len(matching_list_dict[session][4])))
                                                   
    max_dist = GeoDist(3, 10) * max(len(real_data_dict[session]['data']), len(matching_list_dict[session][4])) / (len(real_data_dict[session]['data']) + len(matching_list_dict[session][4]))
    print(100 - dist/max_dist * 100)

    real_path = []
    for i in path[0]:
        real_path.append(matching_list_dict[session][4][i])
                                                   
    synt_path = []
    for i in path[1]:
        synt_path.append(matching_list_dict[session][4][i])
                                                   
    plt.imshow(acc_cost_matrix.T, origin='lower', cmap='gray', interpolation='nearest')
    plt.plot(path[1], path[1], 'w')
    plt.show()
    
    plt.plot(smoothListGaussian(real_data_dict[session]['data']))
    plt.plot(real_path)
    plt.show()
    
    plt.plot(smoothListGaussian(matching_list_dict[session][4][i]))
    plt.plot(synt_path)
    plt.show()

In [ ]:
len(real_sensor_seq)

In [ ]:
len(matching_list)

In [ ]:
A = []
A.append(['a', 'b'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.array([1, 2, 3, 4, 5])
y = np.power(x, 2) # Effectively y = x**2
e = np.array([1.5, 2.6, 3.7, 4.6, 5.5])

plt.errorbar(x, y, e, linestyle='None', marker='^')

plt.show()